In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
pip install seqeval

In [ ]:
!pip install wandb

In [ ]:
pip install datasets

In [ ]:
pip install transformers

In [ ]:
from datasets import load_metric, load_dataset
from transformers import AutoTokenizer
# from transformers import RobertaTokenizer, RobertaModel, RobertaTokenizerFast ## If Using RoBERTa
# from transformers import AlbertTokenizer, AlbertModel, AlbertTokenizerFast ## If using ALBERT
from transformers import BertTokenizer, BertModel, BertTokenizer ## If using BERT
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import numpy as np
import wandb
import transformers
from transformers import EarlyStoppingCallback
from tqdm.notebook import tqdm
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix

In [ ]:
wandb.login()

True

In [ ]:
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
%env TOKENIZERS_PARALLELISM = False ## If you are using multiple GPUs, set this to True
#wandb.init(project="abbDet-roberta-base", entity="<your-wandb-user>")

env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all
env: TOKENIZERS_PARALLELISM=False ## If you are using multiple GPUs, set this to True


In [ ]:
 datasets = load_dataset('ai4bharat/naamapadam','hi')


In [ ]:
label_list = datasets["train"].features[f"ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,tokens,ner_tags
0,"[ईएसआईसी, योजना, के, सामाजिक, सुरक्षा, लाभ, के, कवरेज, को, बढ़ाना, (, ईएसआईसी, 2.0, के, अंतर्गत, )]","[B-ORG, O, O, O, B-ORG, O, O, O, O, O, O, O, O, O, O, O]"
1,"[Samsung, Galaxy, F02s, स्पेसिफिकेशन्स\\xa0]","[O, O, O, O]"
2,"[उन्होंने, कहा, कि, जम्मू, -, कश्मीर, में, विधानसभा, चुनाव, के, कार्यक्रम, की, घोषणा, अमरनाथ, यात्रा, के, समापन, के, बाद, की, जाएगी, ।]","[O, O, O, B-LOC, O, B-LOC, O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,"[इस, घटना, के, बाद, महाराष्ट्र, संगठित, अपराध, नियंत्रण, अधिनियम, (, मकोका, ), के, प्रावधानों, के, तहत, एक, मामला, दर्ज, किया, गया, था, .]","[O, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, B-ORG, O, O, O, O, O, O, O, O, O, O, O, O]"
4,"[इस, अवसर, पर, राष्ट्रपति, ने, अपनी, पुस्तक, के, माध्यम, से, भारत, में, महिलाओं, के, योगदान, को, मान्यता, देने, के, प्रयास, करने, के, लिए, पुस्तक, की, लेखिका, श्रीमती, प्रेम, आहलुवालिया, को, मुबारकबाद, दी, ।]","[O, O, O, O, O, O, O, O, O, O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PER, I-PER, O, O, O, O]"
5,"[मेरा, बेटा, एक, स्पोर्ट्स, पर्सन, है, ,, जो, भारत, की, ओर, से, ओलंपिक, में, पहला, व्यक्तिगत, स्वर्ण, पदक, विजेता, था, .]","[O, O, O, O, O, O, O, O, B-LOC, O, O, O, O, O, O, O, O, O, O, O, O]"
6,"[एक, बार, यह, हो, जाने, के, बाद, ,, आप, d, f, 1, पिन, कोड, देखेंगे, ,, आप, पर्यावरण, अनुभाग, को, भी, देख, सकते, हैं, ।]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,"[इन, संसाधन, केन्द्रों, में, एनआरसी, ,, समाज, विज्ञान, ,, विज्ञान, ,, इंजीनियरिंग, और, टेक्नोलॉजी, ,, डिजाइन, और, मैन्यूफैक्चरिंग, ,, कला, ,, भाषा, ,, शिक्षण, ,, वाणिज्य, ,, प्रबंधन, ,, शिक्षा, नियोजन, और, प्रशासन, ,, लोक, नीति, ,, नेतृत्व, और, शासन, संचालन, ,, पुस्तकालय, और, सूचना, विज्ञान, ,, खगोल, विज्ञान, तथा, खगोल, भौतिकी, ,, निर्धारण, और, मूल्यांकन, ,, अध्यापन, कला, और, शोध, विधि, ,, नैनो, साइंस, ,, इंटरनेट, ऑफ, थिंग्स, जैसे, विभिन्न, अध्ययन, -, विषय, हैं, ।]","[O, O, O, O, B-ORG, O, O, B-ORG, O, O, O, B-ORG, I-ORG, I-ORG, O, B-ORG, I-ORG, I-ORG, O, O, O, O, O, B-ORG, O, B-ORG, O, B-ORG, O, B-ORG, I-ORG, I-ORG, I-ORG, O, B-ORG, I-ORG, O, B-ORG, I-ORG, I-ORG, I-ORG, O, B-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, B-ORG, I-ORG, O, B-ORG, I-ORG, I-ORG, O, O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, O, O, O, O, O, O, O, O]"
8,"[चंडीगढ़, -, हरियाणा, राज्य, चौकसी, ब्यूरो, ने, हरियाणा, वक्फ, बोर्ड, टोहाना, के, संपदा, अधिकारी, ,, मोहम्मद, उस्मान, ,, रैंट, कलेक्टर, मोहम्मद, आजाद, तथा, प्रेम, मेहता, प्रोपर्टी, डीलर, ,, टोहाना, (, मीडियेटर, ), को, टोहाना, ,, जिला, फतेहाबाद, के, निवासी, रघुबीर, सिंह, से, मकान, का, आबंटन, हस्तांतरित, करने, की, एवज, में, 60,000रुपए, की, रिश्वत, लेते, हुए, रंगे, हाथों, पकड़ा, ।]","[B-LOC, O, O, B-ORG, I-ORG, I-ORG, I-ORG, B-ORG, I-ORG, I-ORG, O, O, O, O, O, B-PER, I-PER, O, O, O, B-PER, I-PER, O, B-PER, I-PER, O, O, O, B-LOC, O, O, O, O, B-LOC, O, O, O, O, O, B-PER, I-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
9,"[वाराणसी, /, नयी, दिल्ली, ,, 2, अक्तूबर, (, एजेंसी, )]","[B-LOC, O, B-LOC, I-LOC, O, O, O, O, O, O]"


In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "xlm-roberta-large" ## YOUR LANGUAGE MODEL NAME OR FINETUNED MODEL NAME FOR INFERENCING GOES HERE
batch_size = 4

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)
example = datasets["train"][4]

In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', '▁ज्ञापन', '▁में', '▁कहा', '▁गया', '▁है', '▁कि', '▁सी', 'पी', 'डब्ल्यू', 'डी', '▁की', '▁देख', 'रेख', '▁वाली', '▁1,2', '41', '▁इमारत', 'ों', '▁में', '▁से', '▁223', '▁इमारत', 'ों', '▁में', '▁एल', 'ई', 'डी', '▁ब', 'ल्', 'ब', '▁लगाए', '▁गए', '▁हैं', '▁', 'शेष', '▁230', '▁इमारत', 'ों', '▁में', '▁काम', '▁चल', '▁रहा', '▁है', '▁।', '</s>']


In [ ]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(31, 46)

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 6, 6, 6, 7, 8, 8, 9, 10, 10, 11, 11, 12, 13, 14, 15, 15, 16, 17, 17, 17, 18, 18, 18, 19, 20, 21, 22, 22, 23, 24, 24, 25, 26, 27, 28, 29, 30, None]


In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

46 46


In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) ## For some models, you may need to set max_length to approximately 500.

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/13460 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    report_to="wandb",
    # evaluation_strategy = "epoch", ## Instead of focusing on loss and accuracy, we will focus on the F1 score
    evaluation_strategy ='steps',
    eval_steps = 7000,
    save_total_limit = 3,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=4,
    num_train_epochs=6,
    weight_decay=0.001,
    save_steps=35000,
    push_to_hub=True,
    metric_for_best_model = 'f1',
    load_best_model_at_end=True
)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

<ipython-input-38-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train()

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


In [ ]:
trainer.evaluate() ## Final Evaluation on the validation set

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
trainer.save_model('xlm-r-large-finetuned-ner') ## Change name accordingly

In [ ]:
trainer.push_to_hub()

In [ ]:
true_labels_flat = [item for sublist in true_labels for item in sublist]
true_predictions_flat = [item for sublist in true_predictions for item in sublist]

In [ ]:
# sns.light_palette("seagreen", as_cmap=True)

def plot_cm(y_true, y_pred, figsize=(10,10)):
    cm = confusion_matrix(y_true, y_pred, labels=np.unique(y_true))
    cm_sum = np.sum(cm, axis=1, keepdims=True)
    cm_perc = cm / cm_sum.astype(float) * 100
    annot = np.empty_like(cm).astype(str)
    nrows, ncols = cm.shape
    for i in range(nrows):
        for j in range(ncols):
            c = cm[i, j]
            p = cm_perc[i, j]
            if i == j:
                s = cm_sum[i]
                annot[i, j] = '%.1f%%\n%d/%d' % (p, c, s)
            elif c == 0:
                annot[i, j] = ''
            else:
                annot[i, j] = '%.1f%%\n%d' % (p, c)
    cm = pd.DataFrame(cm, index=np.unique(y_true), columns=np.unique(y_true))
    cm.index.name = 'Actual'
    cm.columns.name = 'Predicted'
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(cm, cmap= "YlGnBu", annot=annot, fmt='', ax=ax).figure.savefig('file.png')

plot_cm(true_labels_flat, true_predictions_flat)